In [7]:
lines = open('input1.txt', 'r')
dial = 50
number_of_zeros = 0
number_of_clicks = 0

for line in lines:
	rotation = int(line[1:]) * (-1 if line[0] == 'L' else 1)
	dial %= (100 if rotation > 0 else -100)

	dial += rotation

	number_of_clicks += abs(dial) // 100
	number_of_zeros += 1 if dial % 100 == 0 else 0

print(f'The number of times the dial stops at zero is {number_of_zeros}')
print(f'The number of times the dial touches zero is {number_of_clicks}')

The number of times the dial stops at zero is 992
The number of times the dial touches zero is 6133


In [4]:
from collections import defaultdict
import math

ranges_string = '11-22,95-115,998-1012,1188511880-1188511890,222220-222224,1698522-1698528,446443-446449,38593856-38593862,565653-565659,824824821-824824827,2121212118-2121212124'
ranges_string = open('input2.txt', 'r').read()
ranges = []
for id_range in ranges_string.split(','):
	start, end = id_range.strip().split('-')
	ranges.append((int(start), int(end)))

invalid_ids = defaultdict(lambda: set())

for start, end in ranges:
	min_digits = len(str(start))
	max_digits = len(str(end))

	for repeats in range(2, max_digits+1):
		min_guess_digits = math.ceil(min_digits / repeats)
		max_guess_digits = math.floor(max_digits / repeats)

		for number_of_digits in range(min_guess_digits, max_guess_digits+1):
			start_range = int(str(start)[:number_of_digits])
			end_range = int(str(end)[:number_of_digits])

			guess = int(start_range)
			while True:
				full_number = int(str(guess) * repeats)
				if start <= full_number <= end:
					invalid_ids[repeats].add(full_number)

				if guess == end_range:
					break
					
				guess = (guess + 1) % 10 ** (number_of_digits)

print(f'The sum of all invalid IDs that repeat twice is {sum(invalid_ids[2])}')
all_invalid_ids = set()
for repeats in invalid_ids:
	all_invalid_ids = all_invalid_ids.union(invalid_ids[repeats])
print(f'The sum of all invalid IDs is {sum(all_invalid_ids)}')

The sum of all invalid IDs that repeat twice is 35367539282
The sum of all invalid IDs is 45814076230
This took 0.0015614032745361328 seconds


In [3]:
import time

before = time.time()
joltages = open('input3.txt', 'r').read().strip().split('\n')

def find_highest_joltages(joltages, number_of_digits):
	highest = []
	for line in joltages:
		digits = [int(c) for c in line.strip()]
		found_digits = []
		starting_index = 0
		for i in range(number_of_digits-1, -1, -1):
			current_range = digits[starting_index:len(digits)-i]
			current_digit = max(current_range)
			found_digits.append(current_digit)
			starting_index += current_range.index(current_digit) + 1
		result = sum(digit * (10 ** index) for index, digit in enumerate(found_digits[::-1]))
		highest.append(result)
	return highest
	
highest = find_highest_joltages(joltages, 2)		
print(f'The sum of the highest joltages when picking two batteries is {sum(highest)}')

highest = find_highest_joltages(joltages, 12)
print(f'The sum of the highest joltages when picking twelve batteries is {sum(highest)}')
after = time

The sum of the highest joltages when picking two batteries is 17158
The sum of the highest joltages when picking twelve batteries is 170449335646486
